In [10]:
pip install aif360


SyntaxError: invalid syntax (<ipython-input-10-5f1e4930a79a>, line 1)

In [26]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [45]:
# Load all necessary packages
import sys
##sys.path.insert(1, "../")  

import pandas as pd
from pandas import read_csv
from pandas import datetime

import numpy as np
np.random.seed(0)

from aif360.datasets import StructuredDataset
from aif360.datasets import GermanDataset
from aif360.metrics import BinaryLabelDatasetMetric

from IPython.display import Markdown, display
from aif360.algorithms.preprocessing.reweighing import Reweighing
#from aif360.algorithms.processing import Reweighing

In [43]:
%matplotlib inline
# Load all necessary packages
import sys
sys.path.append("../")
import numpy as np
from tqdm import tqdm

from aif360.datasets import BinaryLabelDataset
from aif360.datasets import AdultDataset, GermanDataset, CompasDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.algorithms.preprocessing.reweighing import Reweighing
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
        import load_preproc_data_adult, load_preproc_data_german, load_preproc_data_compas
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt

from common_utils import compute_metrics

ModuleNotFoundError: No module named 'common_utils'

In [12]:
dataset_orig = GermanDataset(
    protected_attribute_names=['age'],           # this dataset also contains protected
                                                 # attribute for "sex" which we do not
                                                 # consider in this evaluation
    privileged_classes=[lambda x: x >= 25],      # age >=25 is considered privileged
    features_to_drop=['personal_status','sex'] # ignore sex-related attributes
)

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

In [14]:
datatypes = {
"Age": "float",
"Attrition": "category",
"BusinessTravel": "category",
"DailyRate": "float",
"Department": "category",
"DistanceFromHome": "float",
"Education": "float", ## changed to int
"EducationField": "category",
"EmployeeCount": "float",
"EmployeeNumber": "float",
"EnvironmentSatisfaction": "float", ## changed
"Gender": "category",
"HourlyRate": "float",
"JobInvolvement": "float", ## changed
"JobLevel": "float", #changed
"JobRole": "category",
"JobSatisfaction": "float", ## changed
"MaritalStatus": "category",
"MonthlyIncome": "float",
"MonthlyRate": "float",
"NumCompaniesWorked": "float",
"Over18": "category",
"OverTime": "category",
"PercentSalaryHike": "float",
"PerformanceRating": "float", ## changed
"RelationshipSatisfaction": "float", ## changed
"StandardHours": "float",
"StockOptionLevel": "float",  ## changed
"TotalWorkingYears": "float",
"TrainingTimesLastYear": "float",
"WorkLifeBalance": "float", #changed
"YearsAtCompany": "float",
"YearsInCurrentRole": "float",
"YearsSinceLastPromotion": "float",
"YearsWithCurrManager": "float"
}

df = pd.read_csv('https://raw.githubusercontent.com/skhayden/SMU-Capstone-Age-Bias-in-Predictive-Modeling-/master/WA_Fn-UseC_-HR-Employee-Attrition.csv',low_memory=False)


# Verifying the upload of the data looking at the first 5 rows
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [15]:
## One-hot encoding of categorical features using the dummies function

categorical_fields = list(df.drop(
    columns = ["Age", 
               "DailyRate", 
               "DistanceFromHome",
               "Education", ##
               "EmployeeCount",
               "EmployeeNumber", ## Emp ID
               "EnvironmentSatisfaction", ##
               "HourlyRate", 
               "JobInvolvement", ##
               "JobLevel", ##
               "JobSatisfaction",  ##
               "MonthlyIncome",
               "MonthlyRate",
               "NumCompaniesWorked",
               "PercentSalaryHike",
               "PerformanceRating", ##
               "RelationshipSatisfaction", ##
               "StandardHours",
               "StockOptionLevel", ##
               "TotalWorkingYears",
               "TrainingTimesLastYear",
               "WorkLifeBalance",
               "YearsAtCompany",
               "YearsInCurrentRole",          
               "YearsSinceLastPromotion",     
               "YearsWithCurrManager"        
              ]).select_dtypes(
    include=['category']).columns)


print(categorical_fields)


# Creating dummy variables for their levels
dfOneHot = pd.get_dummies(df, columns = categorical_fields, dtype=float).drop(
    columns = ["Age", 
               "DailyRate", 
               "DistanceFromHome",
               "Education", ##
               "EmployeeCount",
               "EmployeeNumber", ## Emp ID
               "EnvironmentSatisfaction", ##
               "HourlyRate", 
               "JobInvolvement", ##
               "JobLevel", ##
               "JobSatisfaction",  ##
               "MonthlyIncome",
               "MonthlyRate",
               "NumCompaniesWorked",
               "PercentSalaryHike",
               "PerformanceRating", ##
               "RelationshipSatisfaction", ##
               "StandardHours",
               "StockOptionLevel", ##
               "TotalWorkingYears",
               "TrainingTimesLastYear",
               "WorkLifeBalance",
               "YearsAtCompany",
               "YearsInCurrentRole",          
               "YearsSinceLastPromotion",     
               "YearsWithCurrManager" 
              ])


## Verify the OneHot Encoding.
print("Shape after one-hot encoding categorical variables: ", dfOneHot.shape)
print(" ")

print(dfOneHot.info())


[]
Shape after one-hot encoding categorical variables:  (1470, 9)
 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 9 columns):
Attrition         1470 non-null object
BusinessTravel    1470 non-null object
Department        1470 non-null object
EducationField    1470 non-null object
Gender            1470 non-null object
JobRole           1470 non-null object
MaritalStatus     1470 non-null object
Over18            1470 non-null object
OverTime          1470 non-null object
dtypes: object(9)
memory usage: 103.4+ KB
None


In [26]:
nparr = np.array(dfOneHot)
nparr

#starr = StructuredDataset(nparr)

array([['Yes', 'Travel_Rarely', 'Sales', ..., 'Single', 'Y', 'Yes'],
       ['No', 'Travel_Frequently', 'Research & Development', ...,
        'Married', 'Y', 'No'],
       ['Yes', 'Travel_Rarely', 'Research & Development', ..., 'Single',
        'Y', 'Yes'],
       ...,
       ['No', 'Travel_Rarely', 'Research & Development', ..., 'Married',
        'Y', 'Yes'],
       ['No', 'Travel_Frequently', 'Sales', ..., 'Married', 'Y', 'No'],
       ['No', 'Travel_Rarely', 'Research & Development', ..., 'Married',
        'Y', 'No']], dtype=object)

In [21]:
print(dataset_orig)

##print(GermanDataset)

GermanDataset
unprivileged_groups 
dataset_orig_train
dataset_orig
df = dataset_orig.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=True) 

In [42]:
#np.dtype.type(dataset_orig)

type(dataset_orig)

aif360.datasets.german_dataset.GermanDataset

In [22]:

df[::-1]

({'feature_names': ['month',
   'credit_amount',
   'investment_as_income_percentage',
   'residence_since',
   'age',
   'number_of_credits',
   'people_liable_for',
   'status=A11',
   'status=A12',
   'status=A13',
   'status=A14',
   'credit_history=A30',
   'credit_history=A31',
   'credit_history=A32',
   'credit_history=A33',
   'credit_history=A34',
   'purpose=A40',
   'purpose=A41',
   'purpose=A410',
   'purpose=A42',
   'purpose=A43',
   'purpose=A44',
   'purpose=A45',
   'purpose=A46',
   'purpose=A48',
   'purpose=A49',
   'savings=A61',
   'savings=A62',
   'savings=A63',
   'savings=A64',
   'savings=A65',
   'employment=A71',
   'employment=A72',
   'employment=A73',
   'employment=A74',
   'employment=A75',
   'other_debtors=A101',
   'other_debtors=A102',
   'other_debtors=A103',
   'property=A121',
   'property=A122',
   'property=A123',
   'property=A124',
   'installment_plans=A141',
   'installment_plans=A142',
   'installment_plans=A143',
   'housing=A151',
   

In [23]:
print(sys.getsizeof(df))

64


In [24]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.169905
